# ML_25M Deep ML

In [1]:
import torch
import time
import os
import copy
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import itertools
from sklearn.metrics import r2_score, mean_squared_error

def fix_random(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower
fix_random(42)

if not os.path.exists('results'):
    os.mkdir('results')

if not os.path.exists('results/pca'):
    os.mkdir('results/pca')

if not os.path.exists('results/no_pca'):
    os.mkdir('results/no_pca')

pca = None

In [2]:
df = pd.read_csv('dataset.csv')

# PyTorch Device
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print('Using device:', device)

device = torch.device('cpu')

Using device: mps


In [3]:
delete_previous = True

if delete_previous:
    try:
        os.system('rm -rf deepML')
        os.system('rm -rf models')
        os.system('rm -rf best_model.pth')
        os.system('rm -rf best_model_config.json')
    except:
        pass

# Data preprocessing

In [4]:
X = df.drop('rating', axis=1)
Y = df['rating']

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
X_test = X_test.to_numpy()
Y_train = Y_train.to_numpy()
Y_val = Y_val.to_numpy()
Y_test = Y_test.to_numpy()

In [5]:
# # PCA
pca = PCA(n_components=0.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_val = pca.transform(X_val)
X_test = pca.transform(X_test)

In [6]:
print(f'Number of training samples: {X_train.shape[0]}')
print(f'Number of validation samples: {X_val.shape[0]}')
print(f'Number of testing samples: {X_test.shape[0]}')
print(f'\nNumber of features: {X_train.shape[1]}')

val_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(Y_val, dtype=torch.float32)), batch_size=Y_val.shape[0], shuffle=False)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(Y_test, dtype=torch.float32)), batch_size=Y_test.shape[0], shuffle=False)

Number of training samples: 9946
Number of validation samples: 1106
Number of testing samples: 2764

Number of features: 552


# Neural Network

In [7]:
def get_model(input_size, hidden_size, dropout_prob=0, depth=1):
    model = [
        torch.nn.Linear(input_size, hidden_size),
        torch.nn.ReLU(),
        torch.nn.Dropout(dropout_prob)
    ]

    for i in range(depth):
        model.append(torch.nn.Linear(hidden_size, hidden_size))
        model.append(torch.nn.ReLU())
        model.append(torch.nn.Dropout(dropout_prob))

    model.append(torch.nn.Linear(hidden_size, 1))
    return torch.nn.Sequential(*model)


In [8]:
# hyperparameters
hidden_sizes =  [256, 512, 1024]
nums_epochs = [200]
depth = [3, 4, 5]
batch_sizes = [8, 16, 32]
learning_rate = [0.01, 0.001]
step_size_lr_decay = [10, 20]
momentum = [0.9]
dropout_prob = 0.2
patience = 10

hyperparameters = itertools.product(hidden_sizes, depth, nums_epochs, batch_sizes, learning_rate, step_size_lr_decay, momentum)
n_comb = len(hidden_sizes)*len(depth)*len(nums_epochs)*len(batch_sizes)*len(learning_rate)*len(step_size_lr_decay)*len(momentum)
print (f'Number of hyperparameter combinations: {n_comb}')

Number of hyperparameter combinations: 108


In [9]:
def test_model(model, criterion, loader):
    model.eval()
    y_pred = torch.tensor([]).to(device)
    y_true = torch.tensor([]).to(device)

    val_loss = 0.0
    for data in loader:
        data, targets = data[0].to(device), data[1].to(device)
        pred = model(data)
        loss = criterion(pred.squeeze(), targets)
        val_loss += loss.item()
        y_pred = torch.cat((y_pred, pred.squeeze()))
        y_true = torch.cat((y_true, targets.detach()))

    return val_loss / len(loader), y_pred.squeeze(), y_true.squeeze()


In [10]:
# training process
def train_model(model, criterion, optimizer, scheduler, epochs, data_loader, val_loader, device, writer, log_name):
    n_iter = 0

    best_model = None
    best_val_loss = float('inf')
    epochs_since_last_improvement = 0

    start = time.time()

    loss_history = []
    val_loss_history = []

    for epoch in range(epochs):
        model.train()

        start_epoch = time.time()

        loss_train = 0
        for data, targets in data_loader:
            data, targets = data.to(device), targets.to(device)
            y_pred = model(data)
            loss = criterion(y_pred.squeeze(), targets)
            optimizer.zero_grad()
            writer.add_scalar("Loss/train_iter", loss, n_iter)
            loss.backward()
            optimizer.step()
            n_iter += 1
            loss_train += loss.item()

        scheduler.step()
        loss_train /= len(data_loader)
        writer.add_scalar("Loss/train", loss_train, epoch)

        # Compute Val Loss
        val_loss, _ , _ = test_model(model, criterion, val_loader)

        writer.add_scalar("Loss/val", val_loss, epoch)

        loss_history.append(loss_train)
        val_loss_history.append(val_loss)

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = copy.deepcopy(model)
            epochs_since_last_improvement = 0
        elif epochs_since_last_improvement >= patience:
            break
        else:
            epochs_since_last_improvement += 1

        print('Epoch [{}/{}] - {:.2f} seconds - val_loss: {:.6f} - patience: {}'.format(epoch+1,
              epochs, time.time() - start_epoch, val_loss, epochs_since_last_improvement), end='\r')

    print('\nTraining ended after {:.2f} seconds - Best val_loss: {:.6f}'.format(time.time() - start, best_val_loss))

    return best_model, loss_history, val_loss_history

In [11]:
current_iter = 0

best_model = None
best_mse = float('inf')
criterion = torch.nn.MSELoss()
history_loss = []
history_val_loss = []

if os.path.exists('best_model.pth'):
    # read best model config
    with open('best_model_config.json', 'r') as f:
        best_model_config = json.load(f)

    # load best model
    best_model = get_model(X_train.shape[1], best_model_config['hidden_size'], dropout_prob, best_model_config['depth'])
    best_model.load_state_dict(torch.load('best_model.pth'))
    best_model.to(device)
    best_model.eval()

    # evaluate best model
    best_mse, _, _ = test_model(best_model, criterion, test_loader)
    
    print("Best model - MSE: {:.6f}".format(best_mse))

if not os.path.exists('models'):
    os.makedirs('models')

for hidden_size, depth, num_epochs, batch, lr, step_size, momentum in hyperparameters:
    current_iter += 1

    print("\nIterations {}/{} - Training with hidden_size={}, depth={}, num_epochs={}, batch={}, lr={}, step_size={}, momentum={}".format(
        current_iter, n_comb, hidden_size, depth, num_epochs, batch, lr, step_size, momentum))
    
    log_name = "dim:"+str(hidden_size)+"_depth:"+str(depth)+"_batch:" + \
        str(batch)+"_lr:"+str(lr)+"_step_size:" + \
        str(step_size)

    if os.path.exists('deepML/'+log_name):
        print("Model already trained, skipping...")
        continue

    writer = SummaryWriter('deepML/'+log_name)

    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.float32)), batch_size=batch, shuffle=True)

    model = get_model(X_train.shape[1], hidden_size, dropout_prob, depth=depth)
    model = model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

    # train
    model, hl, hvl = train_model(model, criterion, optimizer, scheduler, num_epochs, train_loader, val_loader, device, writer, log_name)

    # validate model on test set
    mse, _, _ = test_model(model, criterion, test_loader)
    writer.add_hparams({'hidden_size': hidden_size, 'depth': depth, 'batch': batch,'lr': lr, 'step_size': step_size, 'momentum': momentum}, {'hparam/mse': mse})

    if mse < best_mse:
        best_mse = mse
        best_model = model
        
        history_loss = hl
        history_val_loss = hvl

        torch.save(model.state_dict(), 'best_model.pth')
        # save config
        with open('best_model_config.json', 'w') as f:
            json.dump({'name':log_name,'hidden_size': hidden_size, 'depth': depth, 'num_epochs': num_epochs, 'batch': batch,
                       'lr': lr, 'step_size': step_size}, f)
            
    print("Model MSE: {:.6f} - Best MSE: {:.6f}".format(mse, best_mse))
    torch.save(model.state_dict(), 'models/'+log_name+'.pth')
    writer.flush()




Iterations 1/108 - Training with hidden_size=256, depth=3, num_epochs=200, batch=8, lr=0.01, step_size=10, momentum=0.9
Epoch [29/200] - 1.42 seconds - val_loss: 0.005351 - patience: 10
Training ended after 52.32 seconds - Best val_loss: 0.005342
Model MSE: 0.005263 - Best MSE: 0.005263

Iterations 2/108 - Training with hidden_size=256, depth=3, num_epochs=200, batch=8, lr=0.01, step_size=20, momentum=0.9
Epoch [48/200] - 1.50 seconds - val_loss: 0.005386 - patience: 10
Training ended after 82.14 seconds - Best val_loss: 0.005319
Model MSE: 0.005174 - Best MSE: 0.005174

Iterations 3/108 - Training with hidden_size=256, depth=3, num_epochs=200, batch=8, lr=0.001, step_size=10, momentum=0.9
Epoch [45/200] - 1.40 seconds - val_loss: 0.007258 - patience: 10
Training ended after 67.43 seconds - Best val_loss: 0.007256
Model MSE: 0.007207 - Best MSE: 0.005174

Iterations 4/108 - Training with hidden_size=256, depth=3, num_epochs=200, batch=8, lr=0.001, step_size=20, momentum=0.9
Epoch [65/

# Neural Network Evalutation

In [12]:
best_model.eval()
_, y_pred, y_true = test_model(best_model, criterion, test_loader)
y_pred = y_pred.squeeze().cpu().detach().numpy()
y_true = y_true.squeeze().cpu().detach().numpy()

In [13]:
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
print("R2: {:.6f} - MSE: {:.6f}".format(r2, mse))

R2: 0.977773 - MSE: 0.004927


In [14]:
results = pd.DataFrame({'loss': [h for h in history_loss], 'val_loss': [h for h in history_val_loss]})

if pca is not None:
    results.to_csv('results/pca/deepML.csv', index=False)
else:
    results.to_csv('results/deepML.csv', index=False)